# 0. 논문에서 모든 데이터 처리는 Python 및 관련 과학 라이브러리, pandas 및 scikit-learn으로 수행됨

In [56]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import f1_score
import soynlp

##### 사용한 모델은 logisitc regression, randomforest, xgboost임으로 사용을 위해 불러왔다.

# 1. 데이터 불러오기 [구조화된 데이터](https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset?select=heart.csv) [텍스트 데이터](https://www.kaggle.com/captainnemo9292/korean-hate-speech-dataset?select=hate_speech_binary_dataset.csv)

In [57]:
structured_data = pd.read_csv("C:/Users/ba874/Desktop/OCL/study/dataset/paper/heart.csv")

In [180]:
text_data = pd.read_csv("C:/Users/ba874/Desktop/OCL/study/dataset/paper/hate_speech_binary_dataset.csv")

In [59]:
structured_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [181]:
text_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190000 entries, 0 to 189999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   문장      189995 non-null  object
 1   혐오 여부   190000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.9+ MB


In [61]:
structured_data["age"].describe()

count    303.000000
mean      54.366337
std        9.082101
min       29.000000
25%       47.500000
50%       55.000000
75%       61.000000
max       77.000000
Name: age, dtype: float64

age : 환자의 나이  
gender : 환자의 성별  
exang : 운동으로 인한 협심증 (1 = 예, 0 = 아니오) //협심증 : 가슴이 좁아진 듯 조이고 뻐근한 통증  
ca : 주요 혈관수 (0-2)  
cp : 가슴 통중 종류 (1 = 전형적인 협심증, 2 = 비정형 협심증, 3 = 비 협심증 통증, 4 = 무증상)  
trtbps : 안정시 혈압 (단위 : mm Hg)
chol : BMI 센서를 통해 가져온 cholestoral (mg/dl)
fbs : 공복 혈당이 120 mg/dl 초과면 1, 아니면 0  
rest_ecg : 휴식 심전도 결과 (0 = 보통, 1 = ST-T파 이상(T 파 반전 및 / 또는 ST 상승 또는 감소> 0.05mV), 2 = Estes의 기준에 따라 가능한 또는 확실한 좌심실 비대 표시)  
thalach : 최대 심박수  
output : 0 = 심장 마비 확률 감소, 1 = 심장 마비 확률 증가

In [88]:
import sklearn

In [125]:
df = sklearn.utils.shuffle(structured_data)

In [126]:
x = df[df.columns.difference(["output"])]

In [127]:
train_data = x.iloc[:202]

In [128]:
test_data = x.iloc[202:]

In [129]:
y = df["output"]

In [130]:
y.sum()

165

In [131]:
train_label = y.iloc[:202]
test_label = y.iloc[202:]

In [132]:
train_label.sum()

109

In [133]:
LR_st = LogisticRegression()

[sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)에 따르면 logistic regression penalty의 default가 l2여서 따로 설정하지 않겠다.

In [134]:
LR_st.fit(train_data, train_label)

C:\Users\ba874\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [135]:
y_predict = LR_st.predict(test_data)

In [136]:
f1_score(test_label, y_predict)

0.8318584070796461

In [137]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score

In [138]:
accuracy_score(test_label, y_predict)

0.8118811881188119

In [139]:
recall_score(test_label, y_predict)

0.8392857142857143

In [140]:
precision_score(test_label, y_predict)

0.8245614035087719

In [141]:
from sklearn.metrics import roc_curve

In [142]:
fpr, tpr, thresholds = roc_curve(test_label, y_predict)

In [143]:
from sklearn.metrics import auc

In [144]:
auc(fpr, tpr)

0.808531746031746

랜포 모델 구현

In [145]:
from sklearn.model_selection import train_test_split

In [146]:
x_train, x_test, y_train, y_test = train_test_split(df[df.columns.difference(["output"])], df["output"], test_size = 0.3)

In [147]:
RF_st = RandomForestClassifier()

In [148]:
RF_st.fit(x_train, y_train)

RandomForestClassifier()

In [149]:
y_pred = RF_st.predict(x_test)

In [150]:
accuracy_score(y_test, y_pred)

0.8241758241758241

In [151]:
recall_score(y_test, y_pred)

0.9090909090909091

In [152]:
precision_score(y_test, y_pred)

0.7692307692307693

In [153]:
f1_score(y_test, y_pred)

0.8333333333333333

In [154]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

In [155]:
auc(fpr, tpr)

0.8268858800773695

xgboost

In [169]:
xgb_rf = xgb.XGBClassifier()

In [170]:
xgb_rf.fit(x_train, y_train)

[17:02:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ba874\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [171]:
pred = xgb_rf.predict(x_test)

In [172]:
accuracy_score(y_test, pred)

0.8241758241758241

In [173]:
recall_score(y_test, pred)

0.8958333333333334

In [174]:
precision_score(y_test, pred)

0.7962962962962963

In [175]:
f1_score(y_test, pred)

0.8431372549019608

In [176]:
fpr, tpr, thresholds = roc_curve(y_test, pred)
auc(fpr, tpr)

0.8200096899224807

In [182]:
df_text = text_data.dropna(axis=0)

In [183]:
df_text.isnull().sum()

문장       0
혐오 여부    0
dtype: int64

In [196]:
corpus = pd.read_csv("C:/Users/ba874/Desktop/OCL/study/dataset/paper/hate_speech_binary_dataset.csv")

In [197]:
corpus

,문장,혐오 여부
0,정말 재밌다 연기도 좋고 디카프리오 짱,1
1,심쿵심쿵 미치네요이수혁땜에 잠 못자겠어요ㅠ자꾸 아른거림ㅠ,1
2,하지만 이니후빨러들은 이런거 관심 하나도 없음,0
3,@착한아이임당 A4 용지 덮고?,0
4,"진짜 평점 믿으면 안된 다는 걸 현실로 보여주는 영화입니다. 네티즌,전문가 하나같이...",1
...,...,...
189995,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,1
189996,케석대 어깨 올라간거봐라 ㅋㅋ,0
189997,@김짜꾸 day and night\n\nround the clock\n\nwitho...,0
189998,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,1


In [198]:
corpus.dropna(axis =0, inplace = True)

In [199]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189995 entries, 0 to 189999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   문장      189995 non-null  object
 1   혐오 여부   189995 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 4.3+ MB


In [204]:
corpus["문장"].to_csv("C:/Users/ba874/Desktop/OCL/study/dataset/paper/corpus.txt", sep="\n", index=False, header = False)

In [206]:
corpus = soynlp.DoublespaceLineCorpus("C:/Users/ba874/Desktop/OCL/study/dataset/paper/corpus.txt")

In [208]:
i = 0
for d in corpus:
    print(i, d)
    i += 1
    if i > 4:
        break

0 정말 재밌다 연기도 좋고 디카프리오 짱
1 심쿵심쿵 미치네요이수혁땜에 잠 못자겠어요ㅠ자꾸 아른거림ㅠ
2 하지만 이니후빨러들은 이런거 관심 하나도 없음
3 @착한아이임당 A4 용지 덮고?
4 진짜 평점 믿으면 안된 다는 걸 현실로 보여주는 영화입니다. 네티즌,전문가 하나같이 영화에서 예술성과 철학만 찾고있으니.. 영화 볼때 평점 안보고 봅니다.


In [209]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)

training was done. used memory 0.774 Gbse memory 0.474 Gb


In [210]:
word_score = word_extractor.extract()

all cohesion probabilities was computed. # words = 126685
all branching entropies was computed # words = 149997
all accessor variety was computed # words = 149997


In [224]:
words = word_extractor.extract()

all cohesion probabilities was computed. # words = 126685
all branching entropies was computed # words = 149997
all accessor variety was computed # words = 149997


In [227]:
from soynlp.tokenizer import MaxScoreTokenizer
scores={w:s.cohesion_forward for w, s in words.items()}
tokenizer=MaxScoreTokenizer(scores=scores)

In [229]:
token = []

In [230]:
for i in corpus:
    token.append(tokenizer.tokenize(i))

In [231]:
token

[['정말', '재밌', '다', '연기', '도', '좋고', '디카프리오', '짱'],
 ['심쿵심쿵', '미치네요이수혁', '땜에', '잠', '못자', '겠어요', 'ㅠ자꾸', '아른거림ㅠ'],
 ['하지만', '이니후빨러들은', '이런', '거', '관심', '하나', '도', '없음'],
 ['@', '착한', '아이', '임당', 'A4', '용지', '덮고', '?'],
 ['진짜',
  '평점',
  '믿으면',
  '안된',
  '다는',
  '걸',
  '현실',
  '로',
  '보여주는',
  '영화',
  '입니다.',
  '네티즌',
  ',',
  '전문가',
  '하나같이',
  '영화',
  '에서',
  '예술',
  '성과',
  '철학',
  '만',
  '찾고',
  '있으니',
  '..',
  '영화',
  '볼때',
  '평점',
  '안',
  '보고',
  '봅니다',
  '.'],
 ['조원진이',
  '비례대표',
  '나',
  '경북',
  '에서',
  '공천',
  '못',
  '받고',
  '도',
  '입다물',
  '면',
  '친중',
  '질히던거',
  '이해할',
  '게'],
 ['후', '!!', '!!', '!!', '!!', '!!', '!', '쩔었다', '..', '.'],
 ['오랜만에',
  '휴가',
  '내서',
  '조카들',
  '데리고',
  '보고',
  '나',
  '왔어요',
  '!!',
  '조카들이',
  '너무',
  '좋아',
  '해서',
  '평점',
  '10',
  '점',
  '남겨용',
  '!!'],
 ['제목만',
  '보고',
  '도',
  '더러운',
  '머리',
  '스탈',
  '떠올랐다',
  '이름보다',
  '아',
  '그더런',
  '새끼',
  '겠지',
  '?',
  '이랬다니까'],
 ['중국', '공산당', '보는', '것', '같다'],
 ['별점', '별점', '별점', '별점', '별점'],
 ['조.센.

In [233]:
len(token)

268790

In [234]:
word2index = {}
bow = []

In [236]:
for i in range(len(token)):
    for j in token[i]:
        if j not in word2index.keys():
            word2index[j] = len(word2index)
            bow.insert(len(word2index)-1, 1)
        else:
            index = word2index.get(j)
            bow[index] = bow[index] + 1

In [237]:
word2index

{'정말': 0,
 '재밌': 1,
 '다': 2,
 '연기': 3,
 '도': 4,
 '좋고': 5,
 '디카프리오': 6,
 '짱': 7,
 '심쿵심쿵': 8,
 '미치네요이수혁': 9,
 '땜에': 10,
 '잠': 11,
 '못자': 12,
 '겠어요': 13,
 'ㅠ자꾸': 14,
 '아른거림ㅠ': 15,
 '하지만': 16,
 '이니후빨러들은': 17,
 '이런': 18,
 '거': 19,
 '관심': 20,
 '하나': 21,
 '없음': 22,
 '@': 23,
 '착한': 24,
 '아이': 25,
 '임당': 26,
 'A4': 27,
 '용지': 28,
 '덮고': 29,
 '?': 30,
 '진짜': 31,
 '평점': 32,
 '믿으면': 33,
 '안된': 34,
 '다는': 35,
 '걸': 36,
 '현실': 37,
 '로': 38,
 '보여주는': 39,
 '영화': 40,
 '입니다.': 41,
 '네티즌': 42,
 ',': 43,
 '전문가': 44,
 '하나같이': 45,
 '에서': 46,
 '예술': 47,
 '성과': 48,
 '철학': 49,
 '만': 50,
 '찾고': 51,
 '있으니': 52,
 '..': 53,
 '볼때': 54,
 '안': 55,
 '보고': 56,
 '봅니다': 57,
 '.': 58,
 '조원진이': 59,
 '비례대표': 60,
 '나': 61,
 '경북': 62,
 '공천': 63,
 '못': 64,
 '받고': 65,
 '입다물': 66,
 '면': 67,
 '친중': 68,
 '질히던거': 69,
 '이해할': 70,
 '게': 71,
 '후': 72,
 '!!': 73,
 '!': 74,
 '쩔었다': 75,
 '오랜만에': 76,
 '휴가': 77,
 '내서': 78,
 '조카들': 79,
 '데리고': 80,
 '왔어요': 81,
 '조카들이': 82,
 '너무': 83,
 '좋아': 84,
 '해서': 85,
 '10': 86,
 '점': 87,
 '남겨용': 88,
 '

In [238]:
bow

[9387,
 10554,
 12209,
 6442,
 16959,
 933,
 75,
 905,
 2,
 1,
 264,
 48,
 4,
 244,
 1,
 1,
 1756,
 1,
 5213,
 4680,
 410,
 3044,
 719,
 3589,
 244,
 1115,
 3,
 30,
 7,
 16,
 4423,
 9822,
 3622,
 18,
 31,
 222,
 1466,
 1790,
 4656,
 385,
 41059,
 1303,
 43,
 8811,
 202,
 121,
 5697,
 268,
 39,
 103,
 5790,
 51,
 170,
 52942,
 174,
 6042,
 6608,
 179,
 32117,
 5,
 49,
 5479,
 16,
 40,
 1930,
 434,
 5,
 4805,
 146,
 1,
 99,
 10748,
 437,
 11758,
 7646,
 13,
 802,
 91,
 59,
 14,
 94,
 56,
 16,
 10294,
 2560,
 3241,
 5373,
 1478,
 1,
 55,
 162,
 808,
 9,
 6,
 2,
 3264,
 1,
 11767,
 1735,
 1,
 2637,
 278,
 2818,
 2580,
 1176,
 189,
 1,
 2,
 326,
 364,
 567,
 55,
 126,
 1,
 118,
 62540,
 11537,
 364,
 3534,
 39,
 3328,
 25569,
 9,
 8214,
 1801,
 993,
 50,
 7,
 1,
 48,
 2,
 1130,
 10963,
 41,
 1,
 1,
 212,
 17254,
 482,
 1589,
 212,
 3866,
 6282,
 322,
 7,
 196,
 311,
 35,
 6763,
 1828,
 237,
 1821,
 1712,
 11,
 1080,
 870,
 17435,
 151,
 192,
 19347,
 2577,
 37,
 13,
 1,
 1437,
 1,
 1819,
 1

In [239]:
len(word2index)

235674

In [240]:
from sklearn.feature_extraction.text import CountVectorizer

In [241]:
bow_vec = CountVectorizer()

In [243]:
bow_vec.fit(corpus)

CountVectorizer()

In [245]:
dtm

<268790x452560 sparse matrix of type '<class 'numpy.int64'>'
	with 1496626 stored elements in Compressed Sparse Row format>

In [249]:
token

[['정말', '재밌', '다', '연기', '도', '좋고', '디카프리오', '짱'],
 ['심쿵심쿵', '미치네요이수혁', '땜에', '잠', '못자', '겠어요', 'ㅠ자꾸', '아른거림ㅠ'],
 ['하지만', '이니후빨러들은', '이런', '거', '관심', '하나', '도', '없음'],
 ['@', '착한', '아이', '임당', 'A4', '용지', '덮고', '?'],
 ['진짜',
  '평점',
  '믿으면',
  '안된',
  '다는',
  '걸',
  '현실',
  '로',
  '보여주는',
  '영화',
  '입니다.',
  '네티즌',
  ',',
  '전문가',
  '하나같이',
  '영화',
  '에서',
  '예술',
  '성과',
  '철학',
  '만',
  '찾고',
  '있으니',
  '..',
  '영화',
  '볼때',
  '평점',
  '안',
  '보고',
  '봅니다',
  '.'],
 ['조원진이',
  '비례대표',
  '나',
  '경북',
  '에서',
  '공천',
  '못',
  '받고',
  '도',
  '입다물',
  '면',
  '친중',
  '질히던거',
  '이해할',
  '게'],
 ['후', '!!', '!!', '!!', '!!', '!!', '!', '쩔었다', '..', '.'],
 ['오랜만에',
  '휴가',
  '내서',
  '조카들',
  '데리고',
  '보고',
  '나',
  '왔어요',
  '!!',
  '조카들이',
  '너무',
  '좋아',
  '해서',
  '평점',
  '10',
  '점',
  '남겨용',
  '!!'],
 ['제목만',
  '보고',
  '도',
  '더러운',
  '머리',
  '스탈',
  '떠올랐다',
  '이름보다',
  '아',
  '그더런',
  '새끼',
  '겠지',
  '?',
  '이랬다니까'],
 ['중국', '공산당', '보는', '것', '같다'],
 ['별점', '별점', '별점', '별점', '별점'],
 ['조.센.

딕셔너리 값을 참고해서 token에 있는것들 index 값으로 바꿔주는 작업을 해야한다  
n gram 적용시켜서 0.1%이상 단어만 남긴다   
단어는 1, 2, 3개 묶는 거까지